# Learning-Based Obstacle Avoidance Policy

## Abstract
This experiment demonstrates a simple learning-based obstacle avoidance system for a mobile robot. The system uses synthetic depth sensor data (front, left, right) and expert demonstrations to train a neural network policy that outputs steering and forward commands. The trained policy is evaluated in a simulated environment where it navigates around obstacles.


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

## 1. Synthetic Dataset Generation
The `ObstacleAvoidDataset` class creates a dataset of sensor readings and corresponding expert control commands:
- **Inputs:** Depth measurements from front, left, and right directions.
- **Targets:** Steering and forward commands generated by an expert rule:
  - If an obstacle is close in front, turn toward the side with more space.
  - Otherwise, go straight forward.

This provides supervised training data for imitation learning.

In [ ]:
# Synthetic dataset: depth from front, left, right (3 values), and command (steer, forward) from expert
class ObstacleAvoidDataset(Dataset):
    def __init__(self, n_samples=2000, seed=0):
        np.random.seed(seed)
        self.depth_front = np.random.uniform(0.5, 5.0, size=(n_samples,))
        self.depth_left = np.random.uniform(0.5, 5.0, size=(n_samples,))
        self.depth_right = np.random.uniform(0.5, 5.0, size=(n_samples,))
        # Expert policy: if front obstacle is close, turn to side with more space
        steer = []
        forward = []
        for f, l, r in zip(self.depth_front, self.depth_left, self.depth_right):
            if f < 1.0:
                if l > r:
                    steer.append(-1.0)  # turn left
                else:
                    steer.append(1.0)   # turn right
                forward.append(0.2)
            else:
                steer.append(0.0)
                forward.append(1.0)
        self.inputs = np.stack([self.depth_front, self.depth_left, self.depth_right], axis=1).astype(np.float32)
        self.targets = np.stack([steer, forward], axis=1).astype(np.float32)

    def __len__(self):
        return len(self.inputs)
    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

## 2. Neural Network Policy
The **SimplePolicy** model is a feedforward neural network that learns to map depth inputs to control outputs:
- 3 input neurons (depth sensors)
- 2 output neurons (steer, forward)
- Hidden layers: 64 and 32 neurons with ReLU activations  
The network predicts steering angle and forward speed from sensor inputs.

---

## 3. Training and Evaluation
The model is trained using **Mean Squared Error (MSE)** loss between predicted and expert commands:
- **Optimizer:** Adam (learning rate = 1e-3)
- **Metrics:** Root Mean Square Error (RMSE) on test data

The training loop iteratively minimizes the imitation loss to replicate expert behavior.

---

In [ ]:
class SimplePolicy(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU(),
            nn.Linear(32, 2)  # steer, forward
        )
    def forward(self, x):
        return self.net(x)

def train_policy(policy, loader, optimizer, device):
    policy.train()
    loss_fn = nn.MSELoss()
    for x, y in loader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        out = policy(x)
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()

def test_policy(policy, loader, device):
    policy.eval()
    total_loss = 0.0
    count = 0
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)
            pred = policy(x)
            total_loss += ((pred - y)**2).sum().item()
            count += y.numel()
    return (total_loss / count)**0.5  # RMSE

## 4. Simulation
After training, the policy is deployed in a simulated 2D environment:
- The robot starts at the origin and moves forward.
- Depth readings are computed synthetically based on the robot’s position relative to a wall.
- The trained policy predicts control actions (steering, forward velocity) at each step.
- The robot’s trajectory is updated accordingly.

This simulates **real-time obstacle avoidance** behavior based on learned sensor-action mapping.


In [ ]:
def simulate_robot(policy, device, n_steps=50):
    # Simulate a robot moving; at each step get synthetic sensor reading, compute action
    pos = np.array([0.0, 0.0])
    heading = 0.0  # angle, radians
    dt = 0.1
    path = [pos.copy()]
    for _ in range(n_steps):
        # fake obstacles: let's say there is a wall at x = 5, so front depth = (5 - x_pos)/cos
        depth_front = (5.0 - pos[0]) / max(np.cos(heading), 0.1)
        depth_left = (5.0 - pos[0]) / max(np.cos(heading + np.pi/4), 0.1)
        depth_right = (5.0 - pos[0]) / max(np.cos(heading - np.pi/4), 0.1)
        depth_front = np.clip(depth_front + np.random.randn()*0.1, 0.1, 10.0)
        depth_left = np.clip(depth_left + np.random.randn()*0.1, 0.1, 10.0)
        depth_right = np.clip(depth_right + np.random.randn()*0.1, 0.1, 10.0)
        inp = torch.tensor([[depth_front, depth_left, depth_right]], dtype=torch.float32, device=device)
        with torch.no_grad():
            out = policy(inp)
        steer, forward = out.cpu().numpy()[0]
        # update heading
        heading += steer * dt
        pos += forward * dt * np.array([np.cos(heading), np.sin(heading)])
        path.append(pos.copy())
    return np.array(path)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = ObstacleAvoidDataset(n_samples=5000)
train_ds, test_ds = torch.utils.data.random_split(dataset, [4000,1000])
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64)

policy = SimplePolicy().to(device)
optimizer = torch.optim.Adam(policy.parameters(), lr=1e-3)
# Train
for epoch in range(10):
    train_policy(policy, train_loader, optimizer, device)
    rmse = test_policy(policy, test_loader, device)
    print(f"Epoch {epoch} RMSE = {rmse:.4f}")
# Simulate
path = simulate_robot(policy, device)
print("Simulated path:", path)

## 5. Discussion
- The trained neural network successfully learns the rule-based obstacle avoidance behavior.
- In simulation, the robot slows down and turns when obstacles appear ahead.
- The model can be extended to handle more complex scenarios, such as noisy sensors, dynamic obstacles, or continuous control tasks.

---

## 6. Key Concepts
- **Imitation Learning:** The policy learns by mimicking an expert’s actions rather than by trial and error.
- **Sensor Fusion:** Multiple depth sensors (front, left, right) provide spatial awareness.
- **End-to-End Control:** The network directly outputs control commands from raw sensor inputs.
